# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse Shipments        Customer  \
0  May 27 2022 10:47AM  238263        19   8070517  Innogenix, LLC   
1  May 27 2022 10:47AM  238263        19   8070518  Innogenix, LLC   
2  May 27 2022 10:47AM  238263        19   8070519  Innogenix, LLC   
3  May 27 2022 10:47AM  238263        19   8070520  Innogenix, LLC   
4  May 27 2022 10:47AM  238263        19   8070521  Innogenix, LLC   
5  May 27 2022 10:47AM  238263        19   8070522  Innogenix, LLC   
6  May 27 2022 10:47AM  238263        19   8070523  Innogenix, LLC   
7  May 27 2022 10:47AM  238263        19   8070524  Innogenix, LLC   
8  May 27 2022 10:47AM  238263        19   8070525  Innogenix, LLC   
9  May 27 2022 10:47AM  238263        19   8070526  Innogenix, LLC   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
27  238255       Released          1
28  238256       Released          1
29  238257       Released          1
30  238262       Released          1
31  238263       Released         23

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Cancelled  Executing  Released
id                                            
238255                NaN        NaN       1.0
238256                NaN        NaN       1.0
238257                NaN        NaN       1.0
238262                NaN        NaN       1.0
238263                NaN        NaN      23.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Cancelled  Executing  Released
id                                            
238159                0.0        0.0       1.0
238165                0.0        5.0       4.0
238168                0.0        0.0       1.0
238171                0.0        0.0       2.0
238177                0.0        5.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Cancelled  Executing  Released
id                                            
238159                0.0          0         1
238165                0.0          5         4
238168                0.0          0         1
238171                0.0          0         2
238177                0.0          5         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Cancelled  Executing  Released
0               238159        0.0          0         1
1               238165        0.0          5         4
2               238168        0.0          0         1
3               238171        0.0          0         2
4               238177        0.0          5         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id  Cancelled Executing Released
0               238159        0.0                  1
1               238165        0.0         5        4
2               238168        0.0                  1
3               238171        0.0                  2
4               238177        0.0         5        1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse  \
0    May 27 2022 10:47AM  238263        19   
23   May 27 2022 10:46AM  238262        19   
24   May 27 2022 10:17AM  238257        19   
25   May 27 2022 10:14AM  238256        17   
26   May 27 2022 10:13AM  238255        10   
27   May 27 2022 10:08AM  238254        10   
62   May 27 2022 10:03AM  238253        20   
64   May 27 2022  9:52AM  238251        20   
93   May 27 2022  9:45AM  238248        10   
131  May 27 2022  9:16AM  238243        10   

                                 Customer  
0                          Innogenix, LLC  
23                         Innogenix, LLC  
24   Bi-Coastal Pharma International, LLC  
25               Amatheon Pharmaceuticals  
26               Amatheon Pharmaceuticals  
27                      ISDIN Corporation  
62                         Exact-Rx, Inc.  
64                      Alumier Labs Inc.  
93                      ISDIN Corporation  
131                     ISDIN Corporation

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse  \
0  May 27 2022 10:47AM  238263        19   
1  May 27 2022 10:46AM  238262        19   
2  May 27 2022 10:17AM  238257        19   
3  May 27 2022 10:14AM  238256        17   
4  May 27 2022 10:13AM  238255        10   
5  May 27 2022 10:08AM  238254        10   
6  May 27 2022 10:03AM  238253        20   
7  May 27 2022  9:52AM  238251        20   
8  May 27 2022  9:45AM  238248        10   
9  May 27 2022  9:16AM  238243        10   

                               Customer  Cancelled Executing Released  
0                        Innogenix, LLC        0.0                 23  
1                        Innogenix, LLC        0.0                  1  
2  Bi-Coastal Pharma International, LLC        0.0                  1  
3              Amatheon Pharmaceuticals        0.0                  1  
4              Amatheon Pharmaceuticals        0.0                  1  
5                     ISDIN Corporation        0.0                 35  
6                        Exact-Rx, Inc.        0.0                  2  
7                     Alumier Labs Inc.        0.0         2       27  
8                     ISDIN Corporation        0.0                 38  
9                     ISDIN Corporation        0.0        12       15

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse  \
0  May 27 2022 10:47AM  238263        19   
1  May 27 2022 10:46AM  238262        19   
2  May 27 2022 10:17AM  238257        19   
3  May 27 2022 10:14AM  238256        17   
4  May 27 2022 10:13AM  238255        10   
5  May 27 2022 10:08AM  238254        10   
6  May 27 2022 10:03AM  238253        20   
7  May 27 2022  9:52AM  238251        20   
8  May 27 2022  9:45AM  238248        10   
9  May 27 2022  9:16AM  238243        10   

                               Customer Released Executing  
0                        Innogenix, LLC       23            
1                        Innogenix, LLC        1            
2  Bi-Coastal Pharma International, LLC        1            
3              Amatheon Pharmaceuticals        1            
4              Amatheon Pharmaceuticals        1            
5                     ISDIN Corporation       35            
6                        Exact-Rx, Inc.        2            
7                     Alumier Labs Inc.       27         2  
8                     ISDIN Corporation       38            
9                     ISDIN Corporation       15        12

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse  \
0  May 27 2022 10:47AM  238263        19   
1  May 27 2022 10:46AM  238262        19   
2  May 27 2022 10:17AM  238257        19   
3  May 27 2022 10:14AM  238256        17   
4  May 27 2022 10:13AM  238255        10   

                               Customer Released Executing  
0                        Innogenix, LLC       23            
1                        Innogenix, LLC        1            
2  Bi-Coastal Pharma International, LLC        1            
3              Amatheon Pharmaceuticals        1            
4              Amatheon Pharmaceuticals        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Date      id Warehouse  \
0  May 27 2022 10:47AM  238263        19   
1  May 27 2022 10:46AM  238262        19   
2  May 27 2022 10:17AM  238257        19   
3  May 27 2022 10:14AM  238256        17   
4  May 27 2022 10:13AM  238255        10   

                               Customer  Released  Executing  
0                        Innogenix, LLC      23.0        NaN  
1                        Innogenix, LLC       1.0        NaN  
2  Bi-Coastal Pharma International, LLC       1.0        NaN  
3              Amatheon Pharmaceuticals       1.0        NaN  
4              Amatheon Pharmaceuticals       1.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse  \
0  May 27 2022 10:47AM  238263        19   
1  May 27 2022 10:46AM  238262        19   
2  May 27 2022 10:17AM  238257        19   
3  May 27 2022 10:14AM  238256        17   
4  May 27 2022 10:13AM  238255        10   

                               Customer  Released  Executing  
0                        Innogenix, LLC      23.0        0.0  
1                        Innogenix, LLC       1.0        0.0  
2  Bi-Coastal Pharma International, LLC       1.0        0.0  
3              Amatheon Pharmaceuticals       1.0        0.0  
4              Amatheon Pharmaceuticals       1.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         2620381      97.0       17.0
12          238210       1.0        0.0
15          476473       2.0        5.0
16          952825       7.0        4.0
17          238256       1.0        0.0
18          238159       1.0        0.0
19          714782      25.0        0.0
20          952840      35.0        7.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  2620381      97.0       17.0
1        12   238210       1.0        0.0
2        15   476473       2.0        5.0
3        16   952825       7.0        4.0
4        17   238256       1.0        0.0
5        18   238159       1.0        0.0
6        19   714782      25.0        0.0
7        20   952840      35.0        7.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10      97.0       17.0
1        12       1.0        0.0
2        15       2.0        5.0
3        16       7.0        4.0
4        17       1.0        0.0
5        18       1.0        0.0
6        19      25.0        0.0
7        20      35.0        7.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   97.0
1        12  Released    1.0
2        15  Released    2.0
3        16  Released    7.0
4        17  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12   15   16   17   18    19    20
Status                                              
Executing  17.0  0.0  5.0  4.0  0.0  0.0   0.0   7.0
Released   97.0  1.0  2.0  7.0  1.0  1.0  25.0  35.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12   15   16   17   18    19    20
0          Executing  17.0  0.0  5.0  4.0  0.0  0.0   0.0   7.0
1           Released  97.0  1.0  2.0  7.0  1.0  1.0  25.0  35.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12   15   16   17   18    19    20
0  Executing  17.0  0.0  5.0  4.0  0.0  0.0   0.0   7.0
1   Released  97.0  1.0  2.0  7.0  1.0  1.0  25.0  35.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()